## Step 3 : Detecting face with mask and without mask

* First, you have to load the model that we created.
* Also, load the cascade classifer for face detection.
* Then we set the camera we want as the default.
* Secondly, we need to label the two probabilities (0 forwith_mask and 1 forwithout_mask).
* After that, we need to set the bounding rectangle color using RGB values. I’ve given RED and GREEN as two colors.

In [1]:
from keras.models import load_model
import cv2
import numpy as np

Using TensorFlow backend.


#### Model no_016 gives me a best loss accracy and validation accuracy
* So, here I'm using model2-016 for detection 

In [2]:
model = load_model('model2-007.model')

face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'with_mask',1:'without_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

* Inside an infinite loop, we are going to read frame by frame from the camera and convert them to grayscale and detect the faces
* it will be run through a for loop to for each face and detect the region of interest, resize and reshape it to 4D since the training network expects 4D input.
* For the model, we are going to use the best model available to get the result. 
* This result consists of the probability (result=[P1, P2]) of the with a mask or without a mask. 
* It will be labeled after that.

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        
        #This result consists of the probability (result=[P1, P2]) of the with a mask or without a mask.
        label=np.argmax(result,axis=1)[0] 
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(
          img, labels_dict[label], 
          (x, y-10),
          cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()